In [1]:
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import csv
import time as t

from datetime import datetime

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
with open('record_linkage.csv', newline='', encoding="utf-8") as f:
    reader = csv.reader(f)
    data = list(reader)
data = np.array(data)
print(data.shape)

(175, 10)


In [4]:
additional_col = np.zeros(shape=[data.shape[0], 3], dtype=object)
print(additional_col.shape)
additional_col[0][0] = 'Position'
additional_col[0][1] = 'Pos_Bat'
additional_col[0][2] = 'Pos_Throw'
player_info = np.append(data, additional_col, 1)
print(player_info.shape)
print(player_info)

(175, 3)
(175, 13)
[['savant_id' 'schedule_id' 'savant_uri' ... 'Position' 'Pos_Bat'
  'Pos_Throw']
 ['0' '547989' 'Abreu_Jose' ... 0 0 0]
 ['1' '660670' 'Acuna_Jr_Ronald' ... 0 0 0]
 ...
 ['171' '657140' 'Wright_Kyle' ... 0 0 0]
 ['172' '573262' 'Yastrzemski_Mike' ... 0 0 0]
 ['173' '592885' 'Yelich_Christian' ... 0 0 0]]


In [5]:
print(player_info[1:,3])

['https://www.mlb.com/player/547989' 'https://www.mlb.com/player/660670'
 'https://www.mlb.com/player/642715' 'https://www.mlb.com/player/542583'
 'https://www.mlb.com/player/645261' 'https://www.mlb.com/player/624413'
 'https://www.mlb.com/player/514888' 'https://www.mlb.com/player/670541'
 'https://www.mlb.com/player/542881' 'https://www.mlb.com/player/462101'
 'https://www.mlb.com/player/571448' 'https://www.mlb.com/player/668227'
 'https://www.mlb.com/player/650333' 'https://www.mlb.com/player/595879'
 'https://www.mlb.com/player/605135' 'https://www.mlb.com/player/641355'
 'https://www.mlb.com/player/605137' 'https://www.mlb.com/player/643217'
 'https://www.mlb.com/player/621244' 'https://www.mlb.com/player/605141'
 'https://www.mlb.com/player/666182' 'https://www.mlb.com/player/669456'
 'https://www.mlb.com/player/453568' 'https://www.mlb.com/player/593428'
 'https://www.mlb.com/player/664761' 'https://www.mlb.com/player/608324'
 'https://www.mlb.com/player/664913' 'https://www.m

In [6]:
path= "chromedriver.exe"

In [7]:
root_path = 'schedule'
runningtime_start=datetime.now()
for i in range(1, player_info.shape[0], 1):

    #driver= webdriver.Chrome(path)
    options = webdriver.ChromeOptions()
    options.headless = True
    driver = webdriver.Chrome(path, chrome_options=options)
    driver.get(player_info[i][3])

    position = driver.find_element(by=By.XPATH,value='/html/body/main/section/header/div/div/ul/li[1]').text
    pos_bt = driver.find_element(by=By.XPATH,value='/html/body/main/section/header/div/div/ul/li[2]').text
    pos_bat = pos_bt.split(': ')[1].split('/')[0]
    pos_throw = pos_bt.split(': ')[1].split('/')[1]
    #print(position, pos_bat, pos_throw)
    player_info[i][10] = position
    player_info[i][11] = pos_bat
    player_info[i][12] = pos_throw
    
    driver.quit()
    if (i-1)%20 == 0:
        print('Processing ... ('+str(i-1)+'/'+str(player_info.shape[0]-1)+') ... ', datetime.now()-runningtime_start)

Processing ... (0/174) ...  0:00:08.106650
Processing ... (20/174) ...  0:03:33.209238
Processing ... (40/174) ...  0:06:55.551738
Processing ... (60/174) ...  0:10:16.436708
Processing ... (80/174) ...  0:13:39.019252
Processing ... (100/174) ...  0:17:04.782834
Processing ... (120/174) ...  0:20:36.352114
Processing ... (140/174) ...  0:24:00.093999
Processing ... (160/174) ...  0:27:27.856509


In [8]:
print(player_info)

[['savant_id' 'schedule_id' 'savant_uri' ... 'Position' 'Pos_Bat'
  'Pos_Throw']
 ['0' '547989' 'Abreu_Jose' ... '1B' 'R' 'R']
 ['1' '660670' 'Acuna_Jr_Ronald' ... 'RF' 'R' 'R']
 ...
 ['171' '657140' 'Wright_Kyle' ... 'P' 'R' 'R']
 ['172' '573262' 'Yastrzemski_Mike' ... 'CF' 'L' 'L']
 ['173' '592885' 'Yelich_Christian' ... 'LF' 'L' 'R']]


In [9]:
np.savetxt('record_linkage_with_pos.csv', player_info, fmt='%s', encoding="utf-8", delimiter=',')